In [22]:
import sys
sys.path.append("..")
from datasets import load_from_disk, load_dataset
from transformers import BertModel, AutoTokenizer, DataCollatorWithPadding, BertForSequenceClassification
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from datasets import load_metric
import pandas as pd
import numpy as np

In [2]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [3]:
sst = load_dataset("sst")

No config specified, defaulting to: sst/default
Reusing dataset sst (/Users/singh.akash/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
sst

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 2210
    })
})

In [5]:
sst2 = load_dataset("SetFit/sst2")

Using custom data configuration SetFit--sst2-c66a8eb4897c1f8f
Reusing dataset json (/Users/singh.akash/.cache/huggingface/datasets/json/SetFit--sst2-c66a8eb4897c1f8f/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
sst2

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 6920
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1821
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 872
    })
})

In [7]:
sst5 = load_dataset("SetFit/sst5")

Using custom data configuration SetFit--sst5-6a5b6ab74c5504bf
Reusing dataset json (/Users/singh.akash/.cache/huggingface/datasets/json/SetFit--sst5-6a5b6ab74c5504bf/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
sst5

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 8544
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2210
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1101
    })
})

In [10]:
sst5.set_format('pandas')
sst5['train']['label'].value_counts()

3    2322
1    2218
2    1624
4    1288
0    1092
Name: label, dtype: int64

In [11]:
sst5['train'].shuffle()[range(10)]

,text,label,label_text
0,in this case zero .,1,negative
1,murderous maids pulls no punches in its depict...,3,positive
2,just how these families interact may surprise ...,2,neutral
3,nair does n't use -lrb- monsoon wedding -rrb- ...,2,neutral
4,"wickedly funny , visually engrossing , never b...",4,very positive
5,an unsuccessful attempt at a movie of ideas .,1,negative
6,the charm of revolution os is rather the way i...,3,positive
7,"lookin ' for sin , american-style ?",1,negative
8,"kwan is a master of shadow , quietude , and ro...",3,positive
9,as warren he stumbles in search of all the emo...,2,neutral


In [12]:
def map_labels(example):
    label = example['label']
    if label == 0 or label ==1:
        return {'ternary_label':0}
    elif label == 2:
        return {'ternary_label':1}
    else:
        return {'ternary_label':2}

In [15]:
sst5.set_format()
sst5 = sst5.map(map_labels)

Loading cached processed dataset at /Users/singh.akash/.cache/huggingface/datasets/json/SetFit--sst5-6a5b6ab74c5504bf/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-8c7e37d2f29eb2cd.arrow
Loading cached processed dataset at /Users/singh.akash/.cache/huggingface/datasets/json/SetFit--sst5-6a5b6ab74c5504bf/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-50375c93b46c256d.arrow
Loading cached processed dataset at /Users/singh.akash/.cache/huggingface/datasets/json/SetFit--sst5-6a5b6ab74c5504bf/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-b46680b54c575eab.arrow


In [16]:
sst5

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'ternary_label'],
        num_rows: 8544
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'ternary_label'],
        num_rows: 2210
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text', 'ternary_label'],
        num_rows: 1101
    })
})

In [25]:
pd.Series(sst5['train']['ternary_label']).value_counts()

2    3610
0    3310
1    1624
dtype: int64

In [26]:
sst5.save_to_disk("../data/sst5")

In [27]:
from huggingface_hub import notebook_login

In [29]:
notebook_login()

In [30]:
from huggingface_hub import create_repo

repo_url = create_repo(name="sst-3", repo_type="dataset")
repo_url

'https://huggingface.co/datasets/artemis13fowl/sst-3'

In [31]:
from huggingface_hub import Repository

repo = Repository(local_dir="sst-3", clone_from=repo_url)


Cloning https://huggingface.co/datasets/artemis13fowl/sst-3 into local empty directory.


cp: ../datasets/sst5: No such file or directory


In [36]:
!cp -r ../data/sst5/ sst-3/

In [37]:
!ls ../data/sst5

dataset_dict.json test              train             validation


In [38]:
repo.push_to_hub()

Upload file train/dataset.arrow:   3%|2         | 32.0k/1.11M [00:00<?, ?B/s]

Upload file test/dataset.arrow:  11%|#         | 32.0k/295k [00:00<?, ?B/s]

Upload file validation/dataset.arrow:  22%|##1       | 32.0k/148k [00:00<?, ?B/s]

remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
To https://huggingface.co/datasets/artemis13fowl/sst-3
   86e7dcc..e9d67b9  main -> main



'https://huggingface.co/datasets/artemis13fowl/sst-3/commit/e9d67b96f6c5754d2bc7a23a7de546a794614b6c'